In [1]:
# All imports here
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier

In [2]:
# Read the train features from CSV
train_features = pd.read_csv("/content/drive/MyDrive/CSCI 5622 - ML/PS4/train-features.csv")

In [3]:
# Read the train labels from CSV
train_labels = pd.read_csv("/content/drive/MyDrive/CSCI 5622 - ML/PS4/train-output.csv")

In [4]:
# Initialize Global variables
scaler = StandardScaler()

In [5]:
train_features.shape

(348607, 55)

In [6]:
train_features.describe()

,Id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53
count,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.00000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000
mean,174303.000000,2959.536458,155.768332,14.110161,269.624006,46.564036,2350.054468,212.140224,223.325863,142.538348,1980.192825,0.448657,0.051336,0.436623,0.063384,0.005295,0.013035,0.008270,0.021442,0.002803,0.011302,0.000181,0.000307,0.001971,0.056132,0.021468,0.051686,0.030057,0.000964,0.000009,0.004819,0.005832,0.003322,0.006902,0.015717,0.001345,0.057165,0.09982,0.036580,0.000806,0.004426,0.001922,0.001667,0.198335,0.051712,0.044586,0.090242,0.077615,0.002834,0.003290,0.000215,0.000565,0.026769,0.023674,0.014919
std,100634.316983,279.980511,111.910643,7.488525,212.688171,58.459698,1558.087024,26.763785,19.782477,38.307210,1323.117587,0.497358,0.220682,0.495968,0.243652,0.072576,0.113423,0.090563,0.144854,0.052865,0.105709,0.013442,0.017517,0.044349,0.230177,0.144940,0.221392,0.170744,0.031031,0.002934,0.069253,0.076143,0.057539,0.082790,0.124378,0.036654,0.232157,0.29976,0.187728,0.028380,0.066382,0.043798,0.040790,0.398746,0.221444,0.206393,0.286529,0.267565,0.053161,0.057266,0.014666,0.023765,0.161409,0.152032,0.121230
min,0.000000,1860.000000,0.000000,0.000000,0.000000,-166.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,87151.500000,2809.000000,58.000000,9.000000,108.000000,7.000000,1106.000000,198.000000,213.000000,119.000000,1024.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,174303.000000,2996.000000,127.000000,13.000000,218.000000,30.000000,1998.000000,218.000000,226.000000,143.000000,1711.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,261454.500000,3164.000000,261.000000,18.0

In [7]:
# Drop feature_28 as it is not useful
def drop_feature_28(df):
  return df.drop(columns=["feature_28"])

In [8]:
# Drop feature 28
train_features = drop_feature_28(train_features)

In [9]:
train_features.columns

Index(['Id', 'feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14',
       'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19',
       'feature_20', 'feature_21', 'feature_22', 'feature_23', 'feature_24',
       'feature_25', 'feature_26', 'feature_27', 'feature_29', 'feature_30',
       'feature_31', 'feature_32', 'feature_33', 'feature_34', 'feature_35',
       'feature_36', 'feature_37', 'feature_38', 'feature_39', 'feature_40',
       'feature_41', 'feature_42', 'feature_43', 'feature_44', 'feature_45',
       'feature_46', 'feature_47', 'feature_48', 'feature_49', 'feature_50',
       'feature_51', 'feature_52', 'feature_53'],
      dtype='object')

In [10]:
# Normalize the numeric features
def normalize_features(df, only_transform=False):

  original_order = df.columns

  # Get the columns to be normalized
  columns = [f"feature_{idx}" for idx in range(0,10)]

  # Normalize Data
  if not only_transform:
    transformed_data = scaler.fit_transform(df[columns])
  else:
    transformed_data = scaler.transform(df[columns])
  
  # Convert to DF
  normalized_df = pd.DataFrame(transformed_data, columns=columns)

  # Concat the df's again
  df = pd.concat([normalized_df, df.loc[:, ~df.columns.isin(columns)]], axis=1)
  df = df[original_order]

  return df

In [11]:
# Normalize features 0 to 9
normalized_train_df = normalize_features(train_features)

In [12]:
normalized_train_df.head(5)

,Id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53
0,0,-0.005488,-0.632366,-0.815938,-1.126646,-0.796516,1.482554,0.742040,0.135177,-0.431730,0.289323,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0.080233,0.806284,-0.682400,0.725834,0.452209,2.415110,-0.416243,1.196724,1.082347,2.483386,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,2,-0.566242,0.270142,0.652978,0.434327,0.999595,-0.965964,0.293672,1.348374,0.325309,-0.040958,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,0.758852,0.189720,0.118827,-0.364967,-0.557035,-0.008379,0.517856,1.196724,0.194785,-0.851167,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,-0.959127,-1.025537,-0.548862,-0.195705,-0.351765,-0.405661,0.368400,-0.319771,-0.327311,3.209702,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
train_labels.head(5)

,Id,Covtype
0,0,1
1,1,2
2,2,5
3,3,1
4,4,2


In [14]:
oversampled_train_df =  normalized_train_df.drop(columns=['Id']).append(normalized_train_df.drop(columns=['Id']))
oversampled_labels = np.concatenate((train_labels['Covtype'],train_labels['Covtype']), axis=0)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(oversampled_train_df, oversampled_labels, test_size=0.10)

In [16]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

627492 69722 627492 69722


# Train the data using RF

In [ ]:
classifer = RandomForestClassifier(n_estimators=50, criterion='entropy', max_depth=50, random_state=0)

In [ ]:
classifer.fit(oversampled_train_df, oversampled_labels)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=50, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
y_pred = classifer.predict(X_test)

In [ ]:
print("F1 score:", f1_score(y_test, y_pred, average='macro'))
print("Accuracy score:", accuracy_score(y_test, y_pred))

F1 score: 0.9946875791574785
Accuracy score: 0.9963139324746851


## Load test data

In [26]:
test_path = "/content/drive/MyDrive/CSCI 5622 - ML/PS4/test-features.csv"

In [27]:
test_df = pd.read_csv(test_path)

In [28]:
test_df = drop_feature_28(test_df)

In [29]:
normalize_test_df = normalize_features(test_df, only_transform=True)

In [30]:
normalize_test_df

,Id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53
0,0,0.476689,-0.909373,-0.415324,-0.952682,-0.642564,1.610917,0.517856,-0.370321,-0.536149,0.334670,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,1,1.037443,0.189720,-1.350089,0.725834,-0.403082,1.331728,0.368400,0.943975,0.351414,0.138164,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,2,0.608842,0.028877,-0.682400,-0.637667,-1.001786,1.398477,0.667312,0.943975,0.012051,-0.695474,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,-1.966341,0.413113,-0.415324,-1.126646,-0.762305,-0.741971,0.106853,1.348374,0.664671,-0.752914,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,0.312392,-0.185580,0.385903,-1.126646,-0.745199,0.630226,1.190408,0.337377,-0.875511,0.715589,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232400,232400,0.758852,-0.748530,-1.216551,0.001768,0.349574,0.180956,0.480492,0.387927,-0.040158,-0.158106,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
232401,232401,0.130236,1.547949,-0.415324,1.529829,0.674585,-0.532098,-0.715155,0.185727,0.795195,0.197116,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
232402,232402,-0.230504,-1.257865,0.118827,-1.126646,-0.676776,-0.320300,-0.229423,-0.724170,-0.118473,-0.489899,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
232403,232403,1.448187,-0.560880,-0.682400,0.034680,-0.060966,-0.241357,0.854132,0.135177,-0.562254,1.008836,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [20]:
true_y_pred = classifer.predict(normalize_test_df.drop(columns=["Id"]))

NameError: ignored

In [21]:
len(true_y_pred)

NameError: ignored

In [ ]:
true_y_pred

array([2, 7, 1, ..., 2, 7, 2])

In [ ]:
result_df = pd.DataFrame(list(zip(test_df.loc[:, "Id"], true_y_pred )), columns=["Id", "Covtype"])

In [ ]:
result_df

,Id,Covtype
0,0,2
1,1,7
2,2,1
3,3,3
4,4,1
...,...,...
232400,232400,1
232401,232401,2
232402,232402,2
232403,232403,7


In [ ]:
result_df.to_csv("/content/drive/MyDrive/CSCI 5622 - ML/PS4/rf_oversampled_submission.csv", index=False)

In [ ]:
result_df['Covtype'].unique()

array([2, 7, 1, 3, 5, 6, 4])

## Gradient Boosting Classifier

In [ ]:
gb_classifier = GradientBoostingClassifier(n_estimators=50, learning_rate=0.01, criterion='mse', random_state=42)

In [ ]:
gb_classifier.fit(X_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='mse', init=None,
                           learning_rate=0.01, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=42, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
gb_pred = gb_classifier.predict(X_test)

In [ ]:
f1_score(y_test, gb_pred, average='macro')

0.2719855468196252

In [ ]:
true_yy = gb_classifier.predict(test_df.drop(columns=["Id"]))

In [ ]:
np.unique(true_yy)

array([1])

## Extra tree classifier

In [ ]:
ext_classifier = ExtraTreesClassifier(n_estimators=50, max_depth=50, random_state=42)

In [ ]:
ext_classifier.fit(X_train, y_train)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=50, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
                     oob_score=False, random_state=42, verbose=0,
                     warm_start=False)

In [ ]:
ext_pred = ext_classifier.predict(X_test)

In [ ]:
f1_score(y_test, ext_pred, average='macro')

0.9067060155630882

In [ ]:
ext_act_pred = ext_classifier.predict(normalize_test_df.drop(columns=["Id"]))

In [ ]:
np.unique(ext_act_pred)

array([1, 2, 3, 4, 5, 6, 7])

In [ ]:
result_df = pd.DataFrame(list(zip(test_df.loc[:, "Id"], ext_act_pred)), columns=["Id", "Covtype"])

In [ ]:
result_df.to_csv("/content/drive/MyDrive/CSCI 5622 - ML/PS4/extra_tree_submission.csv", index=False)

## Xg Boost Classifier 

In [ ]:
xgb_clf = XGBClassifier()

In [ ]:
xgb_clf.fit(X_train, y_train)

KeyboardInterrupt: ignored

In [ ]:
print("Training Score", xgb_clf.score(X_train, y_train))
print("Validation Score", xgb_clf.score(X_test, y_test))

### Try rF hyper parameter tuning

In [ ]:
X_train, y_train = oversampled_train_df, oversampled_labels

In [ ]:
n_estimators = [50, 60, 70, 80, 90]

In [ ]:
from datetime import datetime

In [ ]:
rf_estimator_classifiers = []

start = datetime.now()

for n_estimator in n_estimators:

  print("Training for Estimator size: ", n_estimator)
  classifier = RandomForestClassifier(n_estimators=n_estimator, criterion='entropy', max_depth=50, random_state=0)
  classifier.fit(X_train, y_train)
  
  print("Train score score: ", classifier.score(X_train, y_train))
  print("Validation score: ", classifier.score(X_test, y_test))

  rf_estimator_classifiers.append(classifier)

print("Total time taken", datetime.now()-start)

Training for Estimator size:  50
Train score score:  1.0
Validation score:  1.0
Training for Estimator size:  60
Train score score:  1.0
Validation score:  1.0
Training for Estimator size:  70
Train score score:  1.0
Validation score:  1.0
Training for Estimator size:  80
Train score score:  1.0
Validation score:  1.0
Training for Estimator size:  90
Train score score:  1.0
Validation score:  1.0
Total time taken 0:17:03.840236


In [ ]:
for classifier in rf_estimator_classifiers:
  y_pred = classifier.predict(X_test)
  print("F1 Score:", f1_score(y_test, y_pred, average="macro"))

## Extra Trees

In [17]:
ext_estimators = [80, 85, 90, 95]

In [18]:
from datetime import datetime

In [22]:
ext_estimator_classifiers = []

start = datetime.now()

for n_estimator in ext_estimators:

  print("Training for Estimator size: ", n_estimator)
  classifier = ExtraTreesClassifier(n_estimators=n_estimator, criterion='entropy', max_depth=50, random_state=42)
  classifier.fit(oversampled_train_df, oversampled_labels)
  
  print("Train score score: ", classifier.score(X_train, y_train))
  print("Validation score: ", classifier.score(X_test, y_test))
  y_pred = classifier.predict(X_test)
  print("F1 Score:", f1_score(y_test, y_pred, average="macro"))

  # Comment out
  path = f"/content/drive/MyDrive/CSCI 5622 - ML/PS4/models/ET_oversample_{n_estimator}.sav"
  pickle.dump(classifier, open(path, "wb"))

print("Total time taken", datetime.now()-start)

Training for Estimator size:  80
Train score score:  1.0
Validation score:  1.0
F1 Score: 1.0
Training for Estimator size:  85
Train score score:  1.0
Validation score:  1.0
F1 Score: 1.0
Training for Estimator size:  90
Train score score:  1.0
Validation score:  1.0
F1 Score: 1.0
Training for Estimator size:  95
Train score score:  1.0
Validation score:  1.0
F1 Score: 1.0
Total time taken 0:14:32.000022


## Majority Vote

In [ ]:
combined_classifiers = rf_estimator_classifiers + ext_estimator_classifiers

In [ ]:
overall_predictions = np.zeros((len(normalize_test_df), 5), dtype=int)

In [ ]:
overall_predictions.shape

(232405, 5)

In [ ]:
for idx, classifier in enumerate(rf_estimator_classifiers):
  y_pred = classifier.predict(normalize_test_df.drop(columns=["Id"]))
  overall_predictions[:, idx] = y_pred
  # print("F1 Score:", f1_score(y_test, y_pred, average="macro"))

In [ ]:
overall_predictions.shape

(232405, 5)

In [ ]:
final_predictions = []
for idx, row in enumerate(overall_predictions):
  cp = np.bincount(row).argmax()
  final_predictions.append(cp)

# print("Final F1 score", f1_score(y_test, final_predictions , average='macro') )

In [ ]:
rf_ensemble_result_df = pd.DataFrame(list(zip(normalize_test_df.loc[:, "Id"], final_predictions)), columns=["Id", "Covtype"])

In [ ]:
rf_ensemble_result_df.to_csv("/content/drive/MyDrive/CSCI 5622 - ML/PS4/rf_oversample_ensemble_submission.csv", index=False)

In [21]:
import pickle

In [29]:
paths = [f"/content/drive/MyDrive/CSCI 5622 - ML/PS4/models/ET_oversample_{n}.sav" for n in ext_estimators]

In [30]:
paths

['/content/drive/MyDrive/CSCI 5622 - ML/PS4/models/ET_oversample_80.sav',
 '/content/drive/MyDrive/CSCI 5622 - ML/PS4/models/ET_oversample_85.sav',
 '/content/drive/MyDrive/CSCI 5622 - ML/PS4/models/ET_oversample_90.sav',
 '/content/drive/MyDrive/CSCI 5622 - ML/PS4/models/ET_oversample_95.sav']

In [ ]:
for idx, classifier in enumerate(ext_estimator_classifiers):
  pickle.dump(classifier, open(paths[idx], "wb"))

## Load Models

In [42]:
ext_paths = [f"/content/drive/MyDrive/CSCI 5622 - ML/PS4/models/ET_oversample_{n}.sav" for n in [80, 85, 90, 95]]

In [43]:
rf_paths = [f"/content/drive/MyDrive/CSCI 5622 - ML/PS4/models/RF_oversample_{n}.sav" for n in [70, 80, 90]]

In [44]:
overall_predictions = np.zeros((len(normalize_test_df), 7), dtype=int)

In [45]:
start_index = 0

In [46]:
# Predict the results using RF
for i, path in enumerate(rf_paths):
  classifier = pickle.load(open(path, 'rb'))
  y_pred = classifier.predict(normalize_test_df.drop(columns=["Id"]))
  overall_predictions[:, start_index] = y_pred
  start_index += 1

In [47]:
overall_predictions[0, :]

array([2, 2, 2, 0, 0, 0, 0])

In [48]:
# Predict the results using RF
for i, path in enumerate(ext_paths):
  classifier = pickle.load(open(path, 'rb'))
  y_pred = classifier.predict(normalize_test_df.drop(columns=["Id"]))
  overall_predictions[:, start_index] = y_pred
  start_index += 1

In [49]:
overall_predictions[0, :]

array([2, 2, 2, 2, 2, 2, 2])

In [50]:
final_predictions = []
for idx, row in enumerate(overall_predictions):
  cp = np.bincount(row).argmax()
  final_predictions.append(cp)

In [51]:
rf_ensemble_result_df = pd.DataFrame(list(zip(normalize_test_df.loc[:, "Id"], final_predictions)), columns=["Id", "Covtype"])

In [52]:
rf_ensemble_result_df.to_csv("/content/drive/MyDrive/CSCI 5622 - ML/PS4/rf_ext_7models_oversample_ensemble_submission.csv", index=False)